# Day 19
https://adventofcode.com/2020/day/19

In [1]:
import aocd
data = aocd.get_data(year=2020, day=19)

In [2]:
import regex

##### Part 1: Build regular expression from the provided numbered rules

In [3]:
def read_rules(text):
    return dict(line.split(': ') for line in text.split('\n'))

In [4]:
def read_messages(text):
    return text.split('\n')

In [5]:
def regex_rule(rule, rules):
    if '"' in rule:
        return rule[1]
    parts = rule.split('|')
    if len(parts) > 1:
        return '(?:' + '|'.join(regex_rule(part, rules) for part in parts) + ')'
    parts = parts[0].split(' ')
    return ''.join(regex_rule(rules[part], rules) for part in parts if part)

In [6]:
testdata = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""
rules = read_rules(testdata.split('\n\n')[0])
messages = read_messages(testdata.split('\n\n')[1])
rule = regex_rule(rules['0'], rules)
for message in messages:
    print(message, bool(regex.fullmatch(rule, message)))

ababbb True
bababa False
abbbab True
aaabbb False
aaaabbb False


In [7]:
rulesdata, messagesdata = data.split('\n\n')
rules = read_rules(rulesdata)
messages = read_messages(messagesdata)
rule = regex_rule(rules['0'], rules)
p1 = sum(1 for message in messages if regex.fullmatch(rule, message))
print('Part 1: {}'.format(p1))

Part 1: 184


##### Part 2: Updated rules including some recursion

In [8]:
rules2 = dict(**rules)
rules2.update(**{'8': '42 | 42 8', '11': '42 31 | 42 11 31'})

Let's find which rules use rules 8 and 11.

In [9]:
[(number, content) for number, content in rules2.items()
 if '8' in content.split(' ') or '11' in content.split(' ')]

[('0', '8 11'), ('8', '42 | 42 8'), ('11', '42 31 | 42 11 31')]

So these rules appear only in themselves and in rule 0, which simplifies things somewhat.

- Rule 8 is saying: rule 42 any number of times.
- Rule 11 is saying: any number of 42s followed by the same number of 31s.

So I need to rewrite these, and then run rule 0.

In [10]:
fortytwo = regex_rule(rules2['42'], rules2)
thirtyone = regex_rule(rules2['31'], rules2)
eight = '(?:42)+'.replace('42', fortytwo)
eleven = '((?:4231)|(?:42(?1)31))'.replace('42', fortytwo).replace('31', thirtyone)
zero = eight + eleven
p2 = sum(1 for message in messages if regex.fullmatch(zero, message))
print('Part 2: {}'.format(p2))

Part 2: 389
